In [111]:
from scapy.all import * 
from scapy.all import ICMP, IP
import time
import statistics

Creamos una función ping que envía echo_request al host pasado por parámetro. La función devuelve un diccionario que contiene la siguiente información:

* dst: Host contactado
* response: True si se obtuvo respuesta; False en caso contrario
* rtt_us: RTT en milisegundos (None si no se obtuvo respuesta)
* len: Longitud del paquete recibido (None si no se obtuvo respuesta)
* ttl: Time to live del paquete recibido (None si no se obtuvo respuesta)


In [112]:
def ping(host): 

    # Diccionario de códigos ICMP para respuestas de tipo 3 (Destino inalcanzable)
    codigos = {
        0: "Red de destino inalcanzable",
        1: "Host de destino inalcanzable",
        2: "Protocolo inalcanzable",
        3: "Puerto inalcanzable",
        4: "Fragmentación requerida y DF activado",
        5: "Ruta de origen fallida"}
    
    # Creo diccionario con valores por defecto
    data = {
        "dst": host,
        "response_type": None,
        "rtt_ms": None,
        "len": None,
        "ttl": None
    }
    
    print(f"Pinging {host}...")

    packet = IP(dst=host)/ICMP(type=8) # Echo Request
    
    inicio = time.time()
    response = sr1(packet, timeout=10, verbose=False) # sr1: envío y espero una sola respuesta
    fin = time.time()
    
    if response:

        icmp_layer = response.getlayer(ICMP)

        code = icmp_layer.code

        rtt_ms = (fin - inicio) * 1000 # RTT en milisegundos
        data["rtt_ms"] = rtt_ms
        data["len"] = response.len
        data["ttl"] = response.ttl
        data["response_type"] = response.type

        if response.type == 3: 
            print(f"Destino {host} inalcanzable: " + codigos[code])
        
        elif response.type == 0: # Si la respuesta es echo reply
            print(f"Echo-reply recibida de {host}")

        else:
            print(f"Respuesta ICMP recibida de tipo {response.type}, código {icmp_layer.code}")

    else:
        print(f"No se recibió respuesta de {host}")

    return data

In [122]:
def ping_multiple(host, cantidad_paquetes):

    res = dict()
    sent_packets = cantidad_paquetes
    received_packets = 0
    lost_packets = 0
    total_rtt = 0
    min_rtt = float('inf')
    max_rtt = float('-inf')
    lista_rtts = []
    
    for i in range(cantidad_paquetes):

        resultado = ping(host)

        # print(resultado) # podemos descomentar para ver el resultado de cada ping

        if resultado["response_type"] == 0: 
            received_packets += 1
            total_rtt += resultado["rtt_ms"]
            min_rtt = min(min_rtt, resultado["rtt_ms"])
            max_rtt = max(max_rtt, resultado["rtt_ms"])
            lista_rtts.append(resultado["rtt_ms"])

        else:
            lost_packets += 1 # Si el destino es inalcanzable o no se recibe respuesta, el ping se considera perdido

    res["host"] = host
    res["Paquetes enviados"] = sent_packets
    res["Paquetes recibidos"] = received_packets
    res["Paquetes perdidos"] = lost_packets
    res["Porcentaje de paquetes perdidos"] = (lost_packets / sent_packets) 
    res["RTT total"] = total_rtt if received_packets > 0 else None
    res["RTT mínimo"] = min_rtt if min_rtt != float('inf') else None
    res["RTT máximo"] = max_rtt if max_rtt != float('-inf') else None
    res["RTT promedio"] = total_rtt / received_packets if received_packets > 0 else None
    res["Desvió estándar del RTT promedio"] = statistics.stdev(lista_rtts) if len(lista_rtts) > 1 else None
    
    return res

Contactamos a 5 universidades de continentes distintos. También contactamos a UTDT para ver las diferencias.

In [ ]:
ping_multiple('utdt.edu.ar', 5)


Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
{'dst': 'utdt.edu.ar', 'response_type': 0, 'rtt_ms': 16.39389991760254, 'len': 28, 'ttl': 55}
Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
{'dst': 'utdt.edu.ar', 'response_type': 0, 'rtt_ms': 9.383440017700195, 'len': 28, 'ttl': 55}
Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
{'dst': 'utdt.edu.ar', 'response_type': 0, 'rtt_ms': 10.444164276123047, 'len': 28, 'ttl': 55}
Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
{'dst': 'utdt.edu.ar', 'response_type': 0, 'rtt_ms': 6.869792938232422, 'len': 28, 'ttl': 55}
Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
{'dst': 'utdt.edu.ar', 'response_type': 0, 'rtt_ms': 8.04591178894043, 'len': 28, 'ttl': 55}


{'host': 'utdt.edu.ar',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 51.13720893859863,
 'RTT mínimo': 6.869792938232422,
 'RTT máximo': 16.39389991760254,
 'RTT promedio': 10.227441787719727,
 'Desvió estándar del RTT promedio': 3.7019384254877568}

In [115]:
ping_multiple('mit.edu', 5)

Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu


{'host': 'mit.edu',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 183.41684341430664,
 'RTT mínimo': 34.403085708618164,
 'RTT máximo': 41.126251220703125,
 'RTT promedio': 36.68336868286133,
 'Desvió estándar del RTT promedio': 2.595727034702454}

In [116]:
ping_multiple('ie.edu', 5)

Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu


{'host': 'ie.edu',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 43.84493827819824,
 'RTT mínimo': 7.491111755371094,
 'RTT máximo': 10.917186737060547,
 'RTT promedio': 8.768987655639648,
 'Desvió estándar del RTT promedio': 1.3221477679720761}

In [117]:
ping_multiple('sydney.edu.au', 5)

Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au


{'host': 'sydney.edu.au',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 1563.9467239379883,
 'RTT mínimo': 309.9353313446045,
 'RTT máximo': 314.42880630493164,
 'RTT promedio': 312.78934478759766,
 'Desvió estándar del RTT promedio': 1.7810455006824155}

In [118]:
ping_multiple('cuhk.edu.cn', 5)

Pinging cuhk.edu.cn...
No se recibió respuesta de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn


{'host': 'cuhk.edu.cn',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 4,
 'Paquetes perdidos': 1,
 'Porcentaje de paquetes perdidos': 0.2,
 'RTT total': 1330.493688583374,
 'RTT mínimo': 331.0728073120117,
 'RTT máximo': 335.5364799499512,
 'RTT promedio': 332.6234221458435,
 'Desvió estándar del RTT promedio': 2.0086965350032986}

In [119]:
ping_multiple('alexu.edu.eg', 5)

Pinging alexu.edu.eg...
No se recibió respuesta de alexu.edu.eg
Pinging alexu.edu.eg...
No se recibió respuesta de alexu.edu.eg
Pinging alexu.edu.eg...
No se recibió respuesta de alexu.edu.eg
Pinging alexu.edu.eg...
No se recibió respuesta de alexu.edu.eg
Pinging alexu.edu.eg...
No se recibió respuesta de alexu.edu.eg


{'host': 'alexu.edu.eg',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 0,
 'Paquetes perdidos': 5,
 'Porcentaje de paquetes perdidos': 1.0,
 'RTT total': None,
 'RTT mínimo': None,
 'RTT máximo': None,
 'RTT promedio': None,
 'Desvió estándar del RTT promedio': None}